# An introduction to the `sequence-jacobian` toolkit

This notebook serves as an introduction to the `sequence-jacobian` toolkit and the classes and main functions it provides for solving dynamic general equilibrium models in sequence space.

This introduction will cover the following topics:
0. `sequence-jacobian` preliminaries
    - The `sequence-jacobian` toolkit directory structure
    - Importing `sequence-jacobian`
1. How do `Block` objects work?
2. The main functions of `sequence-jacobian`
    - `steady_state`
    - `get_G`
    - `td_map`

The notebook accompanies the working paper by Auclert, Bardóczy, Rognlie, Straub (2019): "Using the Sequence-Space Jacobian to Solve and Estimate Heterogeneous-Agent Models". Please see the [Github repository](https://github.com/shade-econ/sequence-jacobian) for more information and code.

Also, be sure to check out the [Real Business Cycle (RBC) Notebook](https://github.com/shade-econ/sequence-jacobian/blob/develop-cai/notebooks/rbc.ipynb), which this notebook partially draws from, to see the Sequence-Jacobian solution method applied to a standard model.

# `sequence-jacobian` preliminaries

##  0.1 The `sequence-jacobian' directory structure 

We will only illustrate a smaller subset of the directory structure for the sake of clarity.

```
sequence-jacobian/
│
├──notebooks/
│  ├─ intro_to_sequence_jacobian.ipynb
│  └─ rbc.ipynb
│  
└──sequence_jacobian/
   ├─ blocks/
   │    ├─ simple
   │    ├─ het
   │    ├─ helper
   │    └─ solved
   │
   ├─ models/
   │    ├─ rbc
   │    ├─ krusell_smith
   │    └─ hank
   │
   ├─ utilities/
   │    ├─ discretize
   │    ├─ graph
   │    └─ devtools
   │
   ├─ jacobian
   ├─ nonlinear
   └─ steady_state
```

The `sequence-jacobian` repository contains a few directories at the top-level.

The notable ones for the purposes of this introduction are:
- `notebooks`: where example notebooks demonstrate the `sequence-jacobian` functionality on well-known models
- `sequence_jacobian`: the main python package containing the source code.

Within `sequence_jacobian`, we have:
- Modules implementing the main steps of the solution method: `jacobian`, `nonlinear`, `steady_state`
- Sub-directories containing:
    - `blocks`: definitions of the various `Block` objects, which wrap model equations
    - `models`: implemented models as collections of `Block` objects
    - `utilities`: useful functions that support user in debugging their code and that improve workflow within the `sequence-jacobian` source code

##  0.2 Importing the `sequence-jacobian` code

Given the present directory structure of the toolkit, we will need to import the `sequence_jacobian` module from the directory containing the one we are currently in. Thus, we will use the `sys` package to add the directory containing this one to the path, allowing us to access the `sequence_jacobian` module from the top-level.

In [29]:
import sys
sys.path.append("..")

import numpy as np

import sequence_jacobian as sj
from sequence_jacobian import simple

# 1 How do `Block` objects work?

## 1.1 `SimpleBlock`

The simplest python class that `sequence-jacobian` provides is called the `SimpleBlock`.

These blocks are intended for use in defining the model equations that are solely functions of *aggregate* variables and parameters as in the equilibrium conditions from the representative firm problem in the real business cycle model:

$$
r_t + \delta = \alpha Z_t \left(\frac{K_{t-1}}{L_t} \right)^{\alpha-1}
\\
w_t = (1-\alpha) Z_t \left(\frac{K_{t-1}}{L_t} \right)^{\alpha}
$$

`SimpleBlock`, like the other main `Block` classes, has a few **attributes** that describe its contents:
- `inputs`
- `outputs`
- `.f`

A custom **constructor**, in the form of a python decorator, `@simple`.

And three main **methods**:
- `.ss`, which is used in the steady state computation
- `.td`, which is used in the transitional dynamics
- `.jac`, which is used in the Jacobian computation

In [4]:
@simple
def my_first_simple_block(a, b, c):
    d = a + b
    e = b * c
    return d, e

The attributes describe what variables `my_first_simple_block` takes as inputs, what variables it outputs, and provide a way to invoke the function it is defined over.

In [5]:
my_first_simple_block.inputs

{'a', 'b', 'c'}

In [6]:
my_first_simple_block.outputs

{'d', 'e'}

In [7]:
my_first_simple_block.f

<function __main__.my_first_simple_block(a, b, c)>

Hence, we can directly invoke the function that `my_first_simple_block` is defined over as if it were a simple python function

In [12]:
my_first_simple_block.f(1, 2, 3)

(3, 6)

Also as seen above, the custom constructor requires no additional arguments and can simply be used by annotating the function on the line above with the decorator `@simple` as is standard in python.

Because dynamic general equilibrium models are, by definition, *dynamic*, typically the equations that define such models contain time displacements.

We can represent the equations $d_t = a_{t+1} + b_t$ and $e_t = b_{t+1} * c_{t-1}$ as the following `SimpleBlock`, using Dynare-like syntax

In [9]:
@simple
def my_first_dynamic_block(a, b, c):
    d = a(1) + b
    e = b(1) * c(-1)
    return d, e

Notice here, we cannot simply invoke `my_first_dynamic_block` as if it were a simple python function because `a(1)` is not standard python syntax. Under the hood, `sequence-jacobian` catches these inputs and wraps them in special classes to handle time displacement prior to invoking the function to capture information relevant for the steady state, jacobian, etc. which we will get to.

In [13]:
my_first_dynamic_block.f(1, 2, 3)

TypeError: 'int' object is not callable

In general, we would not advise the user to directly invoke the `.f` attribute but rather work with the main methods of the `SimpleBlock`: `.ss`, `.td`, `.jac`.

As mentioned above, the main way the user should interface with `SimpleBlock`s directly (which itself will typically be unnecessary once we get to the main functions in the `sequence-jacobian` toolkit) is through the methods `.ss`, `.td`, and `.jac`.

These methods utilize the information encoded in the `SimpleBlock` both by the algebraic relationships written in the variable definitions but also in the time displacements, as in `my_first_dynamic_block` above.

For example, in `.ss` the dynamic equation $d_t = a_{t+1} + b_t$ would become $d^* = a^* + b^*$, where the $*$ variables indicate their steady state values, whereas in `.td` or `.jac`, the information from the dynamic equation is retained so that transitions or Jacobians are calculated with the correct time displacements.

The main functions in the `sequence-jacobian` toolkit provide the main handling of these time displacements, so we will not elaborate on this further here.

Below are some other examples of more complicated equations contained in `SimpleBlock`s that demonstrate their flexibility.

We are also accommodate the use of more arbitrary functions called on the variables that enter into `SimpleBlock`s, with `.apply()` for single argument functions and `apply_function()` for functions with multiple arguments.

**Note**: As of now `apply_function()` has more limited functionality, only working within the context of transitional dynamics and *not* Jacobian calculation. Either reference the source code for more information, or otherwise proceed with caution!

In [33]:
from sequence_jacobian import apply_function

@simple
def heres_a_block(a, b, c):
    d = (a(-1) ** b)/c
    e = -d(+1)
    return d, e

@simple
def heres_another_one(a, b, c):
    d = a(-1).apply(np.log)
    e = b(+1) + c(+1)
    return d, e

def foo(a, b, c):
    return np.sin(a) + b ** c

@simple
def and_another(a, b, c):
    d = apply_function(foo, a, b, c)
    return d

# And finally a more realistic one
@simple
def firm(K, L, Z, alpha, delta):
    r = alpha * Z * (K(-1) / L) ** (alpha-1) - delta
    w = (1 - alpha) * Z * (K(-1) / L) ** alpha
    return r, w

## 1.2 HetBlock

The next class that `sequence-jacobian` provides is the `HetBlock`. While it is nice to be able to work with aggregate variables, one of the main selling points of the `sequence-jacobian` toolkit is that it provides a fast, general framework for solving heterogeneous agent models. 

Hence, the `HetBlock` provides a class that is compatible with `SimpleBlock`s and other kinds of blocks to be described later on to be able to define the main components of a dynamic general equilibrium model in a piecemeal fashion.

`HetBlock`s 

# 2 The main functions of `sequence-jacobian`